## CausalANOVA Implementation

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import class_FM as FM
import rpy2.robjects as ro

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from sklearn import preprocessing
from itertools import combinations

In [29]:
utils = importr('utils')
utils.chooseCRANmirror(ind=1)
utils.install_packages('FindIt')

R[write to console]: also installing the dependencies ‘nloptr’, ‘lme4’, ‘RcppEigen’, ‘arm’, ‘glmnet’


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/nloptr_2.0.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2219877 bytes (2.1 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

checking whether the C++ compiler works... yes
checking for C++ compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether the compiler supports GNU C++... yes
checking whether arm64-apple-darwin20.0.0-clang++ -std=gnu++17 accepts -g... yes
checking for arm64-apple-darwin20.0.0-clang++ -std=gnu++17 option to enable C++11 features... none needed
checking how to run the C++ preprocessor... arm64-apple-darwin20.0.0-clang++ -std=gnu++17 -E
checking whether the compiler supports GNU C++... (cached) yes
checking whether arm64-apple-darwin20.0.0-clang++ -std=gnu++17 accepts -g... (cached) yes
checking for arm64-apple-darwin20.0.0-clang++ -std=gnu++17 option to enable C++11 features... (cached) none needed
checking for pkg-config... no
checking for cmake... no

------------------ CMAKE NOT FOUND --------------------

CMake was not found on the PATH. Please install CMak

ERROR: configuration failed for package ‘nloptr’
* removing ‘/opt/miniconda3/envs/causal/lib/R/library/nloptr’
* installing *source* package ‘RcppEigen’ ...
** package ‘RcppEigen’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
using C++ compiler: ‘clang version 17.0.6’
using SDK: ‘MacOSX14.4.sdk’


arm64-apple-darwin20.0.0-clang++ -std=gnu++17 -I"/opt/miniconda3/envs/causal/lib/R/include" -DNDEBUG  -I'/opt/miniconda3/envs/causal/lib/R/library/Rcpp/include' -D_FORTIFY_SOURCE=2 -isystem /opt/miniconda3/envs/causal/include -mmacosx-version-min=11.0 -I/opt/miniconda3/envs/causal/include   -I../inst/include -fPIC  -ftree-vectorize -fPIC -fstack-protector-strong -O2 -pipe -stdlib=libc++ -fvisibility-inlines-hidden -fmessage-length=0 -isystem /opt/miniconda3/envs/causal/include -fdebug-prefix-map=/Users/runner/miniforge3/conda-bld/r-base-split_1709218193821/work=/usr/local/src/conda/r-base-4.3.3 -fdebug-prefix-map=/opt/miniconda3/envs/causal=/usr/local/src/conda-prefix  -c RcppEigen.cpp -o RcppEigen.o


In file included from RcppEigen.cpp:22:
In file included from ../inst/include/RcppEigen.h:25:
In file included from ../inst/include/RcppEigenForward.h:28:
In file included from ../inst/include/Eigen/Dense:1:
In file included from ../inst/include/Eigen/Core:171:
../inst/include/Eigen/src/Core/MathFunctions.h:743:16: error: no member named 'Rlog1p' in namespace 'std'; did you mean 'log1p'?
  743 |     using std::log1p;
      |           ~~~~~^~~~~
      |                log1p
/opt/miniconda3/envs/causal/lib/R/include/Rmath.h:73:15: note: expanded from macro 'log1p'
   73 | #define log1p Rlog1p
      |               ^
/opt/miniconda3/envs/causal/bin/../include/c++/v1/cmath:448:9: note: 'log1p' declared here
  448 | using ::log1p _LIBCPP_USING_IF_EXISTS;
      |         ^
1 error generated.
make: *** [/opt/miniconda3/envs/causal/lib/R/etc/Makeconf:200: RcppEigen.o] Error 1
ERROR: compilation failed for package ‘RcppEigen’
* removing ‘/opt/miniconda3/envs/causal/lib/R/library/RcppEigen’
ERR

<rpy2.rinterface_lib.sexp.NULLType object at 0x1070c4e50> [0]

In [26]:
# Import your required R packages
base = importr("base")
findit = importr("FindIt")

PackageNotInstalledError: The R package "FindIt" is not installed.

### Initialize factorial model and sample dataset

In [17]:
n = 1000
k = 3
degree = 3
sigma = 0.1
sparsity = 0.5

fm = FM.FactorialModel(
    n=n,
    k=k,
    degree=degree,
    sigma=sigma,
    sparsity=sparsity,
    beta_seed=42,
)
t, y = fm.sample(seed=0)

### Fit OLS model with sample data

In [18]:
T = preprocessing.PolynomialFeatures(
    degree=degree, interaction_only=True, include_bias=True,
).fit_transform(t)
m = sm.OLS(y, T)
results = m.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                 2.093e+04
Date:                Mon, 18 Mar 2024   Prob (F-statistic):               0.00
Time:                        08:33:25   Log-Likelihood:                 881.00
No. Observations:                1000   AIC:                            -1746.
Df Residuals:                     992   BIC:                            -1707.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0085      0.009      0.964      0.3

### Fit CausalANOVA model with sample dataset

In [21]:
# Convert NumPy arrays to pandas DataFrame
data = pd.DataFrame(t.astype("str"), columns=["t1", "t2", "t3"], dtype="category")
data["outcome"] = y

# Convert pandas DataFrame to R data.frame
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data = ro.conversion.py2rpy(data)

ro.globalenv["r_data"] = r_data

fac_level = ro.IntVector([2, 2, 2])
ro.globalenv["fac_level"] = fac_level

_ = ro.r(
    """
fit <- CausalANOVA(formula=outcome ~ t1 + t2 + t3, family="gaussian", data=r_data, nway=3)
"""
)
print(ro.r("summary(fit)"))

R[write to console]: Error in CausalANOVA(formula = outcome ~ t1 + t2 + t3, family = "gaussian",  : 
  could not find function "CausalANOVA"

R[write to console]: In addition: 
R[write to console]: There were 24 warnings (use warnings() to see them)
R[write to console]: 



RRuntimeError: Error in CausalANOVA(formula = outcome ~ t1 + t2 + t3, family = "gaussian",  : 
  could not find function "CausalANOVA"
